## Configuration and Exploratory Analysis of the SPLINK Model

This notebook contains the configuration and exploratory analysis for using the [SPLINK](https://moj-analytical-services.github.io/splink/index.html) model to run a record linkage algorithm.
The goal is to execute this model after receiving results from [OpenSearch](https://opensearch.org/).

In general, the idea is to perform the following steps when someone queries the `GET /v1/production-locations/` endpoint with the name and address of the production location:

1. First, query the OpenSearch cluster's `production-locations` index using [hybrid search](https://opensearch.org/docs/latest/vector-search/ai-search/hybrid-search/index/).

2. After obtaining the search results, use the [real-time record linkage](https://moj-analytical-services.github.io/splink/demos/examples/duckdb/real_time_record_linkage.html) capability of the SPLINK library to generate confidence scores for potential matches.


In [10]:
# add default imports
import opensearchpy as osp
import pandas as pd
import splink as spl

## Load dataset

In [11]:
locs_df = pd.read_csv("production_locations.csv")
locs_df = locs_df.drop(columns=[
    "created_at",
    "updated_at",
    "potential_match_score",
    "potential_match_os_id",
    "found_index"
])
locs_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1118341 entries, 0 to 1118340
Data columns (total 10 columns):
 #   Column                  Non-Null Count    Dtype  
---  ------                  --------------    -----  
 0   os_id                   1118341 non-null  object 
 1   name                    1118341 non-null  object 
 2   address                 1118339 non-null  object 
 3   lat                     1118341 non-null  float64
 4   lng                     1118341 non-null  float64
 5   geo_hash                1118341 non-null  object 
 6   country_code            1118331 non-null  object 
 7   found_score             1118341 non-null  float64
 8   geocoded_location_type  1027110 non-null  object 
 9   geocoded_address        1027110 non-null  object 
dtypes: float64(3), object(7)
memory usage: 85.3+ MB
